In [3]:
%load_ext autoreload
%autoreload 2

import os
import time
import sys

from utilities2014 import *

sys.path.append('/home/yuncong/project/opencv-2.4.9/release/lib/python2.7/site-packages')
import cv2

from joblib import Parallel, delayed

from collections import defaultdict, Counter
from itertools import combinations, chain, product

import matplotlib.pyplot as plt
%matplotlib inline

import cPickle as pickle

In [4]:
from enum import Enum

class PolygonType(Enum):
    CLOSED = 'closed'
    OPEN = 'open'
    TEXTURE = 'textured'
    TEXTURE_WITH_CONTOUR = 'texture with contour'
    DIRECTION = 'directionality'

In [5]:
dm = DataManager(generate_hierarchy=False, stack='RS141', resol='x5', section=4)
dm._load_image()

texton_hists = dm.load_pipeline_result('texHist', 'npy')
segmentation = dm.load_pipeline_result('segmentation', 'npy')
n_superpixels = len(np.unique(segmentation)) - 1
textonmap = dm.load_pipeline_result('texMap', 'npy')
n_texton = len(np.unique(textonmap)) - 1
neighbors = dm.load_pipeline_result('neighbors', 'pkl')
sp_properties = dm.load_pipeline_result('spProps', 'npy')
segmentation_vis = dm.load_pipeline_result('segmentationWithText', 'jpg')

# texture_map = texton_hists[segmentation]
# texture_map[~dm.mask] = np.nan * np.ones((n_texton,))
# dm.save_pipeline_result(texture_map, 'textureMap', 'npy')
texture_map = dm.load_pipeline_result('textureMap', 'npy')

coherence_map = dm.load_pipeline_result('coherenceMap', 'npy')
eigenvec_map = dm.load_pipeline_result('eigenvecMap', 'npy')

origPosCanvas = dm.load_pipeline_result('originPosOnCanvas', 'npy')

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


In [6]:
# labeling_name = 'RS141_0001_yuncong_08282015013756'
labeling_name = 'RS141_0004_yuncong_08282015023125'
labeling = dm.load_labeling(labeling_name='_'.join(labeling_name.split('_')[-2:]))

In [7]:
thetas = np.linspace(-np.pi/4, np.pi/4, 9)
n_theta = len(thetas)
Rs = [np.array([[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]]) for theta in thetas]

In [27]:
from scipy.spatial.distance import cdist

def vertices_to_normals(vertices):
    
    n = vertices.shape[0]

    # compute normal direction of each vertex
    D = cdist(vertices, vertices)
    nn = D.argsort(axis=1)[:,:3]
    vertice_normals = np.empty((n, 2))
    for i, neighborhood in enumerate(nn):
        X = vertices[neighborhood]
        Xc = X - X.mean(axis=0)
        U,S,V = np.linalg.svd(np.dot(Xc.T, Xc))
        tangent_dir = U[:,0]
        vertice_normals[i] = np.array([tangent_dir[1], -tangent_dir[0]])
        if vertice_normals[i][0] < 0: # make sure x-component is positive, i.e. normal points rightwards
            vertice_normals[i] = -vertice_normals[i]
    return vertice_normals


def find_in_polygon(vertices, x, y):

    from matplotlib.path import Path

    X, Y = np.meshgrid(x, y)  # X, Y are 2D ndarrays
    XY = np.dstack((X, Y))
    XY_flat = XY.reshape((-1, 2))

    mpath = Path(vertices) # the vertices of the polygon
    mask_flat = mpath.contains_points(XY_flat)
    mask = mask_flat.reshape(X.shape).copy()

    return mask.astype(np.bool)

    
def compute_landmark_descriptors(group):
    
    n_polygon = len(group)
        
    all_polygon_types, all_polygon_vertices_global = zip(*group)
                
    edge_indices = [i for i in range(n_polygon)
                   if all_polygon_types[i] == PolygonType.TEXTURE_WITH_CONTOUR \
                            or all_polygon_types[i] == PolygonType.CLOSED\
                           or all_polygon_types[i] == PolygonType.OPEN]
    texture_indices = [i for i in range(n_polygon)
                   if all_polygon_types[i] == PolygonType.TEXTURE_WITH_CONTOUR \
                            or all_polygon_types[i] == PolygonType.TEXTURE]
    
    res = [{'bbox':None, 'polygons':[{'type': all_polygon_types[i]} for i in range(n_polygon)]} 
           for _ in range(n_theta)]
        
    all_vertices = np.vstack(all_polygon_vertices_global)
    
    xmin, ymin = all_vertices.min(axis=0).astype(np.int)
    xmax, ymax = all_vertices.max(axis=0).astype(np.int)
    centroid_global = all_vertices.mean(axis=0).astype(np.int)
    centroid_local = centroid_global - [xmin, ymin]
    lm_texture_template = texture_map[ymin:ymax+1, xmin:xmax+1]

    lm_box_shape = [xmax - xmin + 1, ymax - ymin + 1]

    texture_sample_radius = 5
    int_rs = np.arange(-texture_sample_radius, 0)
    ext_rs = np.arange(1, texture_sample_radius+1)
    
    all_polygon_vertice_normals = map(vertices_to_normals, all_polygon_vertices_global)
    
    striation_indices = [i for i in range(n_polygon) if all_polygon_types[i] == PolygonType.DIRECTION]
    if len(striation_indices) > 0:
        striation_sample_points = np.vstack(all_polygon_vertices_global[i] for i in striation_indices).astype(np.int)
        striation_sample_vecs = eigenvec_map[striation_sample_points[:,1], striation_sample_points[:,0]]

    
    for theta_i in range(n_theta):
        
        all_polygon_vertices_rotated_global = [np.dot(vertices - centroid_global, Rs[theta_i].T).astype(np.int) + centroid_global
                                               for vertices in all_polygon_vertices_global]
                
        all_polygon_vertices_rotated_global_stacked = np.vstack(all_polygon_vertices_rotated_global)
        all_polygon_vertices_rotated_global_xymin = all_polygon_vertices_rotated_global_stacked.min(axis=0)
        all_polygon_vertices_rotated_global_xymax = all_polygon_vertices_rotated_global_stacked.max(axis=0)
                
        centroid_rotated_local = centroid_global - all_polygon_vertices_rotated_global_xymin
        
        rotated_bbox_shape = all_polygon_vertices_rotated_global_xymax - all_polygon_vertices_rotated_global_xymin + 1
        
        res[theta_i]['bbox'] = np.array([rotated_bbox_shape[0], rotated_bbox_shape[1], 
                                 centroid_rotated_local[0], centroid_rotated_local[1],
                                        centroid_global[0], centroid_global[1],
                                        xmin, ymin, xmax, ymax])
        
        all_polygon_vertices_rotated_local = [vertices_rotated_global - all_polygon_vertices_rotated_global_xymin
                                              for vertices_rotated_global in all_polygon_vertices_rotated_global]
        
        all_polygon_vertices_rotated_canvas = [vertices_rotated_global + origPosCanvas 
                                               for vertices_rotated_global in all_polygon_vertices_rotated_global]

        
#         if len(edge_indices) > 0:
            
#             all_polygon_vertice_normals_rotated = [np.dot(vertice_normals, Rs[theta_i].T)
#                                                    for vertice_normals in all_polygon_vertice_normals]
            
        for i in edge_indices:
            polygon = res[theta_i]['polygons'][i]

            polygon['boundary_normals'] = np.dot(all_polygon_vertice_normals[i], Rs[theta_i].T)
            polygon['boundary_vertices_local'] = all_polygon_vertices_rotated_local[i]
            polygon['boundary_vertices_global'] = all_polygon_vertices_rotated_global[i]
            polygon['boundary_vertices_canvas'] = all_polygon_vertices_rotated_canvas[i]

            int_texture_sample_xs = (all_polygon_vertices_global[i][:,0][:,None] + np.outer( polygon['boundary_normals'][:,0], int_rs)).astype(np.int)
            int_texture_sample_ys = (all_polygon_vertices_global[i][:,1][:,None] + np.outer( polygon['boundary_normals'][:,1], int_rs)).astype(np.int)
            polygon['boundary_int_texture'] = texture_map[int_texture_sample_ys, int_texture_sample_xs].mean(axis=1)

            ext_texture_sample_xs = (all_polygon_vertices_global[i][:,0][:,None] + np.outer( polygon['boundary_normals'][:,0], ext_rs)).astype(np.int)
            ext_texture_sample_ys = (all_polygon_vertices_global[i][:,1][:,None] + np.outer( polygon['boundary_normals'][:,1], ext_rs)).astype(np.int)
            polygon['boundary_ext_texture'] = texture_map[ext_texture_sample_ys, ext_texture_sample_xs].mean(axis=1)
                                        
    
        for i in striation_indices:
            polygon = res[theta_i]['polygons'][i]

            polygon['striation_points_local'] = all_polygon_vertices_rotated_local[i].astype(np.int)
            
            # original
            striation_sample_points = all_polygon_vertices_global[i].astype(np.int)
            striation_sample_vecs = eigenvec_map[striation_sample_points[:,1], striation_sample_points[:,0]]
            
            polygon['striation_vecs'] = np.dot(striation_sample_vecs, Rs[theta_i].T)
            polygon['striation_points_global'] = all_polygon_vertices_global[i]
            polygon['striation_points_canvas'] = all_polygon_vertices_rotated_canvas[i]      

        # rotated templates
        
        if len(texture_indices) > 0:
        
            rotated_texTemplate = np.nan * np.ones((rotated_bbox_shape[1], rotated_bbox_shape[0], n_texton))

            ys, xs = np.mgrid[:rotated_bbox_shape[1], :rotated_bbox_shape[0]]
            a = np.dot(np.linalg.inv(Rs[theta_i]), (np.c_[xs.flat, ys.flat] - centroid_rotated_local).T).T

            xss = (a[:,0] + centroid_local[0]).astype(np.int)
            yss = (a[:,1] + centroid_local[1]).astype(np.int)

            valid = (yss < lm_texture_template.shape[0]) & (yss >= 0) & (xss < lm_texture_template.shape[1]) & (xss >= 0)
            rotated_texTemplate[ys.flat[valid], xs.flat[valid]] = lm_texture_template[yss[valid], xss[valid]].copy()

            for i in texture_indices:
                polygon = res[theta_i]['polygons'][i]

                polygon_mask = find_in_polygon(all_polygon_vertices_rotated_local[i], 
                                               range(rotated_bbox_shape[0]), range(rotated_bbox_shape[1]))

                polygon['textured_area_mask'] = polygon_mask
                
#                 polygon['textured_area_template'] = np.nan * np.ones_like(rotated_texTemplate)
#                 polygon['textured_area_template'][polygon_mask] = rotated_texTemplate[polygon_mask].copy()
#                 textured_pixels_ys, textured_pixels_xs = np.where(~np.isnan(polygon['textured_area_template'][...,0]))

#                 textured_area_template = np.nan * np.ones_like(rotated_texTemplate)
#                 textured_area_template[polygon_mask] = rotated_texTemplate[polygon_mask]
                
#                 polygon['textured_area_template_sparse'] = textured_area_template[::10, ::10].copy()
            
                polygon['textured_area_texton_histogram'] = np.nanmean(rotated_texTemplate[polygon_mask], axis=0)
        
#                 textured_pixels_ys, textured_pixels_xs = np.where(~np.isnan(textured_area_template[...,0]))
#                 textured_pixels_xys_global = np.c_[textured_pixels_xs, textured_pixels_ys] + all_polygon_vertices_rotated_global_xymin
#                 polygon['textured_area_pixels_global'] = textured_pixels_xys_global
                polygon['textured_area_vertices_global'] = all_polygon_vertices_rotated_global[i]
                polygon['textured_area_vertices_canvas'] = all_polygon_vertices_rotated_canvas[i]

                
        if len(edge_indices) > 0:
            res[theta_i]['boundary_normals'] = np.vstack(res[theta_i]['polygons'][i]['boundary_normals'] for i in edge_indices)
            res[theta_i]['boundary_vertices_local'] = np.vstack(res[theta_i]['polygons'][i]['boundary_vertices_local'] for i in edge_indices)
            res[theta_i]['boundary_vertices_global'] = np.vstack(res[theta_i]['polygons'][i]['boundary_vertices_global'] for i in edge_indices)
            res[theta_i]['boundary_vertices_canvas'] = np.vstack(res[theta_i]['polygons'][i]['boundary_vertices_canvas'] for i in edge_indices)
            res[theta_i]['boundary_int_texture'] = np.vstack(res[theta_i]['polygons'][i]['boundary_int_texture'] for i in edge_indices)
            res[theta_i]['boundary_ext_texture'] = np.vstack(res[theta_i]['polygons'][i]['boundary_ext_texture'] for i in edge_indices)
        else:
            res[theta_i]['boundary_vertices_local'] = []
            res[theta_i]['boundary_vertices_global'] = []

        if len(striation_indices) > 0:
            res[theta_i]['striation_points_local'] = np.vstack(res[theta_i]['polygons'][i]['striation_points_local'] for i in striation_indices)
            res[theta_i]['striation_vecs'] = np.vstack(res[theta_i]['polygons'][i]['striation_vecs'] for i in striation_indices)
            res[theta_i]['striation_points_global'] = np.vstack(res[theta_i]['polygons'][i]['striation_points_global'] for i in striation_indices)
            res[theta_i]['striation_points_canvas'] = np.vstack(res[theta_i]['polygons'][i]['striation_points_canvas'] for i in striation_indices)
        else:
            res[theta_i]['striation_points_local'] = []
            res[theta_i]['striation_points_global'] = []
            
        if len(texture_indices) > 0:
#             res[theta_i]['textured_area_template'] = np.sum(np.nan_to_num(res[theta_i]['polygons'][i]['textured_area_template']) for i in texture_indices)

            mask = np.logical_or.reduce([res[theta_i]['polygons'][i]['textured_area_mask'] for i in texture_indices])
            res[theta_i]['textured_area_mask'] = mask
            
#             res[theta_i]['textured_area_template'] = np.nan * np.ones_like(rotated_texTemplate)
#             res[theta_i]['textured_area_template'][mask] = rotated_texTemplate[mask]

            res[theta_i]['textured_area_masks_and_texhists'] = [(res[theta_i]['polygons'][i]['textured_area_mask'],
                                                                 res[theta_i]['polygons'][i]['textured_area_texton_histogram'])
                                                                for i in texture_indices]

#             textured_area_template_sparse = np.nan * np.ones_like(rotated_texTemplate)
#             textured_area_template_sparse[mask] = rotated_texTemplate[mask]
#             res[theta_i]['textured_area_template_sparse'] = textured_area_template_sparse[::10, ::10].copy()
            
#             res[theta_i]['textured_area_pixels_global'] = np.vstack(res[theta_i]['polygons'][i]['textured_area_pixels_global'] for i in texture_indices)
            res[theta_i]['textured_area_vertices_global'] = np.vstack(res[theta_i]['polygons'][i]['textured_area_vertices_global'] for i in texture_indices)
        else:
            res[theta_i]['textured_area_vertices_global'] = []
        
    return res

In [24]:
vertices_name = {PolygonType.OPEN: 'boundary_vertices_global',
                 PolygonType.CLOSED: 'boundary_vertices_global',
                 PolygonType.DIRECTION: 'striation_points_global',
                 PolygonType.TEXTURE: 'textured_area_vertices_global',
                 PolygonType.TEXTURE_WITH_CONTOUR: 'textured_area_vertices_global'
                }

In [25]:
def compute_landmark_descriptors_helper(label, group):
    descriptor = compute_landmark_descriptors(group)
    with open('/home/yuncong/csd395/landmark_descriptors/%s_landmark_descriptors_%d.pkl'%(labeling_name,label), 'w') as f:
        pickle.dump(descriptor, f)        
#     return label, descriptor

In [18]:
# b = time.time()

# Parallel(n_jobs=16)(delayed(compute_landmark_descriptors_helper)(label, group)
#                     for label, group in labeling['final_polygons'].iteritems())

# print time.time() - b

In [28]:
centriods_global = {}
landmark_vertices_global = {}
landmark_polygon_vertices_global = {}

for label, group in labeling['final_polygons'].iteritems():
    print label
    descriptor = compute_landmark_descriptors(group)

    with open('/home/yuncong/csd395/landmark_descriptors/%s_landmark_descriptors_%d.pkl'%(labeling_name,label), 'w') as f:
        pickle.dump(descriptor, f)
        
    d = descriptor[4]
    
    centriods_global[label] = descriptor[0]['bbox'][4:6]
    landmark_vertices_global[label] = [d['boundary_vertices_global'], 
                                       d['textured_area_vertices_global'], 
                                       d['striation_points_global']]
    landmark_polygon_vertices_global[label] = [(p['type'], p[vertices_name[p['type']]]) for p in d['polygons']]

0
1
2
3
4
5
6
7
8
9
10
11
12
14
15
16


In [31]:
with open('/home/yuncong/csd395/landmark_descriptors/%s_landmark_centroids_global.pkl'%labeling_name, 'w') as f:
    pickle.dump(centriods_global, f)

In [32]:
with open('/home/yuncong/csd395/landmark_descriptors/%s_landmark_vertices_global.pkl'%labeling_name, 'w') as f:
    pickle.dump(landmark_vertices_global, f)

In [33]:
with open('/home/yuncong/csd395/landmark_descriptors/%s_landmark_polygon_vertices_global.pkl'%labeling_name, 'w') as f:
    pickle.dump(landmark_polygon_vertices_global, f)

In [ ]:
# import tables as T
# import numpy as N

# ra = N.array([(3.14159,'some string',[1,2,3,4,5],N.arange(0,10,0.1))],
# dtype = "f8,S20,(5,)i4,(100,)f8")

# f = T.openFile("test.h5", "w")
# t = f.createTable(f.root, 'table', ra)
# print "table metadata-->", repr(t)
# print "table data-->", t[:]

# f.close()